# Pre-Processing Route File


In [1]:
import sys
import pathlib
import os


def find_root(path):
    if os.path.split(path)[-1] != "amazon-routing-challenge":
        return find_root(os.path.split(path)[0])
    return path


ROOT = find_root(pathlib.Path().absolute())
sys.path.append(ROOT)


In [2]:
import json
import pandas as pd

In [14]:
data_path = os.path.join(
    ROOT,
    "data",
    "almrrc2021-data-training",
    "model_build_inputs",
)


# test_data_path = os.path.join(
#     ROOT,
#     "data",
#     "almrrc2021-data-evaluation",
#     "model_eval_inputs",
# )

route_file = os.path.join(data_path, "route_data.json")
package_file = os.path.join(data_path, "package_data.json")
sequence_file = os.path.join(data_path, "actual_sequences.json")
tt_file = os.path.join(data_path, "travel_times.json")


## Create the Route DataFrame


In [4]:
route_df = pd.DataFrame.from_records(
    (
        {
            "stop_id": k,
            "route_id": route_id,
            "station_code": route_data["station_code"],
            "departure_datetime": route_data["date_YYYY_MM_DD"]
            + " "
            + route_data["departure_time_utc"],
            "executor_capacity_cm3": route_data["executor_capacity_cm3"],
            "route_score": route_data["route_score"],
            **v,
        }
        for route_id, route_data in json.load(
                open(route_file, "r")
            ).items()
        for k, v in route_data["stops"].items()
    )
)


In [5]:
route_df['departure_datetime'] = pd.to_datetime(route_df['departure_datetime'], utc=True)
route_df['departure_datetime'] = route_df['departure_datetime'].dt.tz_convert('EST')

## Add in the Package Information


In [6]:
package_df = pd.DataFrame.from_records(
    (
        {
            "route_id": k,
            "stop_id": s,
            "package_id": p,
            "status": p_data["scan_status"],
            "time_window_start": p_data["time_window"]["start_time_utc"],
            "time_window_end": p_data["time_window"]["end_time_utc"],
            "planned_service_time": p_data["planned_service_time_seconds"],
            "width": p_data["dimensions"]["width_cm"],
            "depth": p_data["dimensions"]["depth_cm"],
            "height": p_data["dimensions"]["height_cm"],
        }
        for k, k_data in json.load(open(package_file)).items()
        for s, s_data in k_data.items()
        for p, p_data in s_data.items()
    )
)


### Merge Route and Package Data

In [7]:
package_df['has_time_window'] = package_df['time_window_start'].notnull()
package_df['volume'] = package_df['width'] * package_df['depth'] * package_df['height']

package_df_grouped = package_df.groupby(['route_id', 'stop_id']).agg(
    {
        'package_id': 'nunique',
        'has_time_window': 'first',
        'planned_service_time': 'sum',
        'width': 'mean',
        'depth': 'mean',
        'height': 'mean',
        'volume': 'sum',
    }
).reset_index()

In [8]:
package_df_grouped.head()

,route_id,stop_id,package_id,has_time_window,planned_service_time,width,depth,height,volume
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AD,3,False,177.9,22.466667,30.066667,9.30,18700.228
1,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AF,1,True,27.0,27.400000,30.000000,3.00,2466.000
2,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AG,2,False,90.0,26.700000,37.450000,10.15,19134.644
3,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,BA,1,False,38.0,16.800000,24.400000,1.00,409.920
4,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,BE,4,False,167.2,19.775000,33.725000,9.15,29776.244


In [9]:
route_df = route_df.merge(package_df_grouped, on=['route_id', 'stop_id'], how='left')

# force garbage collection
import gc
del package_df_grouped
del package_df
gc.collect()

0

## Open the Actual Sequence Data


In [15]:
sequence_data = json.load(
    open(sequence_file, "r")
)


In [16]:
# drop the actual level from the sequence data
sequence_data = {k: v["actual"] for k, v in sequence_data.items()}


### Sort the Individual Routes by their Actual Order


In [17]:
# route_df["lat_lon"] = list(zip(route_df.lat, route_df.lng))
route_df["order"] = route_df[["route_id", "stop_id"]].apply(
    lambda x: sequence_data[x[0]][x[1]], raw=True, axis=1
)

route_df = route_df.sort_values(
    by=["route_id", "order"],
)
route_df.reset_index(drop=True, inplace=True)
route_df.head()


,stop_id,route_id,station_code,departure_datetime,executor_capacity_cm3,route_score,lat,lng,type,zone_id,package_id,has_time_window,planned_service_time,width,depth,height,volume,order
0,VE,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27 11:02:10-05:00,3313071.0,High,34.007369,-118.143927,Station,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,TG,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27 11:02:10-05:00,3313071.0,High,34.088467,-118.284521,Dropoff,A-2.2A,7.0,False,184.8,25.342857,34.157143,12.871429,107970.047,1
2,GP,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27 11:02:10-05:00,3313071.0,High,34.088709,-118.284839,Dropoff,A-2.2A,1.0,False,34.0,19.100000,30.500000,10.200000,5942.010,2
3,HT,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27 11:02:10-05:00,3313071.0,High,34.088717,-118.286484,Dropoff,A-2.2A,2.0,False,145.0,18.800000,29.700000,6.350000,8010.523,3
4,AG,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27 11:02:10-05:00,3313071.0,High,34.089727,-118.285530,Dropoff,A-2.1A,2.0,False,90.0,26.700000,37.450000,10.150000,19134.644,4


In [18]:
# sanity check to assert that the sequence data is correct
assert all(
    route_df.groupby("route_id").apply(
        lambda x: x["order"].is_monotonic_increasing and x["order"].is_unique
    )
)


## Apply the Amazon Travel Times


In [19]:
tt_json = json.loads(open(os.path.join(data_path, "travel_times.json"), "r").read())

In [21]:
def apply_travel_time(df: pd.DataFrame) -> pd.DataFrame:
    # add on the return back to the station. This shifts the travel time so that the 
    # row contains the time to get to the rows stop_id from the previous row 
    # and the service time for that stop
    df.loc[df.index.max() + 1] = df.iloc[0]
    df.reset_index(drop=True, inplace=True)
    df["prior_id"] = df["stop_id"].shift(1, fill_value=df["stop_id"].iloc[-1])
    df["travel_time"] = df[["route_id", "stop_id", "prior_id"]].apply(
        lambda x: tt_json[x[0]][x[1]][x[2]], axis=1, raw=True
    )
    return df


In [22]:
route_df["prior_id"] = ""
route_df["travel_time"] = 0

route_df = route_df.groupby("route_id").apply(apply_travel_time).reset_index(drop=True)


In [23]:
route_df['norm_service_time'] = route_df['sum_service_time'] / route_df['num_packages']


### Add Both Service Time and Travel Time to Get Total Time


In [26]:
route_df["time_total"] = route_df["travel_time"] + route_df["planned_service_time"]


In [28]:
def apply_total_time(df: pd.DataFrame) -> pd.DataFrame:
    df["cumulative_time"] = df["time_total"].cumsum()
    _time = df['departure_datetime'] + pd.to_timedelta(df['cumulative_time'], unit='s')
    # df['time_of_day'] =  _time.apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
    df['time_of_day'] =  _time.apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
    return df


route_df["cumulative_time"] = 0
route_df["time_of_day"] = 0
route_df['departure_datetime'] = pd.to_datetime(route_df['departure_datetime'])
route_df = route_df.groupby("route_id").apply(apply_total_time)

In [31]:
route_df.head()

,stop_id,route_id,station_code,departure_datetime,executor_capacity_cm3,route_score,lat,lng,type,zone_id,...,width,depth,height,volume,order,prior_id,travel_time,time_total,cumulative_time,time_of_day
0,VE,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27 11:02:10-05:00,3313071.0,High,34.007369,-118.143927,Station,NaN,...,NaN,NaN,NaN,NaN,0,VE,0.0,NaN,NaN,NaN
1,TG,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27 11:02:10-05:00,3313071.0,High,34.088467,-118.284521,Dropoff,A-2.2A,...,25.342857,34.157143,12.871429,107970.047,1,VE,1651.4,1836.2,1836.2,41566.0
2,GP,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27 11:02:10-05:00,3313071.0,High,34.088709,-118.284839,Dropoff,A-2.2A,...,19.100000,30.500000,10.200000,5942.010,2,TG,17.6,51.6,1887.8,41617.0
3,HT,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27 11:02:10-05:00,3313071.0,High,34.088717,-118.286484,Dropoff,A-2.2A,...,18.800000,29.700000,6.350000,8010.523,3,GP,35.5,180.5,2068.3,41798.0
4,AG,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27 11:02:10-05:00,3313071.0,High,34.089727,-118.285530,Dropoff,A-2.1A,...,26.700000,37.450000,10.150000,19134.644,4,HT,89.6,179.6,2247.9,41977.0


### Drop Station, Sequence, and Datetime Information and Save

In [36]:
route_df = route_df.loc[route_df['type'].str.startswith('D')].reset_index(drop=True)
# route_df.drop(columns=['cumulative_time', 'time_total', 'travel_time', 'prior_id', 'order', 'zone_id'], inplace=True)

#### Assert the there are No Duplicate Entries

In [39]:
assert len((route_df['route_id'] + " " + route_df['stop_id']).unique()) == len(route_df)

In [37]:
route_df[route_df.columns.difference(['cumulative_time', 'time_total', 'travel_time', 'prior_id', 'order', 'zone_id', 'departure_datetime'])].head()

,depth,executor_capacity_cm3,has_time_window,height,lat,lng,package_id,planned_service_time,route_id,route_score,station_code,stop_id,time_of_day,type,volume,width
0,34.157143,3313071.0,False,12.871429,34.088467,-118.284521,7.0,184.8,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,High,DLA3,TG,41566.0,Dropoff,107970.047,25.342857
1,30.500000,3313071.0,False,10.200000,34.088709,-118.284839,1.0,34.0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,High,DLA3,GP,41617.0,Dropoff,5942.010,19.100000
2,29.700000,3313071.0,False,6.350000,34.088717,-118.286484,2.0,145.0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,High,DLA3,HT,41798.0,Dropoff,8010.523,18.800000
3,37.450000,3313071.0,False,10.150000,34.089727,-118.285530,2.0,90.0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,High,DLA3,AG,41977.0,Dropoff,19134.644,26.700000
4,32.866667,3313071.0,True,9.900000,34.090901,-118.284257,3.0,254.1,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,High,DLA3,QM,42297.0,Dropoff,21141.936,22.700000


In [38]:
route_df[route_df.columns.difference(['cumulative_time', 'time_total', 'travel_time', 'prior_id', 'order', 'zone_id', 'departure_datetime'])].to_pickle(
    os.path.join(data_path, 'route_df_merged_augmented.pkl'),
    protocol=4
)